In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/raw/used_cars.csv")

df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [ ]:
df.info()
df.describe(include="all")

<class 'pandas.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   brand         4009 non-null   str  
 1   model         4009 non-null   str  
 2   model_year    4009 non-null   int64
 3   milage        4009 non-null   str  
 4   fuel_type     3839 non-null   str  
 5   engine        4009 non-null   str  
 6   transmission  4009 non-null   str  
 7   ext_col       4009 non-null   str  
 8   int_col       4009 non-null   str  
 9   accident      3896 non-null   str  
 10  clean_title   3413 non-null   str  
 11  price         4009 non-null   str  
dtypes: int64(1), str(11)
memory usage: 376.0 KB


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
count,4009,4009,4009.000000,4009,3839,4009,4009,4009,4009,3896,3413,4009
unique,57,1898,NaN,2818,7,1146,62,319,156,2,1,1569
top,Ford,M3 Base,NaN,"110,000 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,A/T,Black,Black,None reported,Yes,"$15,000"
freq,386,30,NaN,16,3309,52,1037,905,2025,2910,3413,39
mean,NaN,NaN,2015.515590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,6.104816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1974.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2012.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2017.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2020.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# check data types for price and milage
df[["price", "milage"]].dtypes

# convert price to numeric
df["price"] = (
    df["price"]
    .astype(str)
    .str.replace(r"[^0-9.]", "", regex=True)
)

df["price"] = pd.to_numeric(df["price"], errors="coerce")

# convert milage to numeric
df["milage"] = (
    df["milage"]
    .astype(str)
    .str.replace(r"[^0-9]", "", regex=True)
)

df["milage"] = pd.to_numeric(df["milage"], errors="coerce")

# drop rows with missing price or milage
df = df.dropna(subset=["price", "milage"])

# convert price and milage to integers
df["price"] = df["price"].astype(int)
df["milage"] = df["milage"].astype(int)

df[["price", "milage"]].dtypes

price     int64
milage    int64
dtype: object

In [ ]:
# check fuel types
df["fuel_type"].value_counts(dropna=False)


fuel_type
Gasoline          3309
Hybrid             194
NaN                170
E85 Flex Fuel      139
Diesel             116
–                   45
Plug-In Hybrid      34
not supported        2
Name: count, dtype: int64

In [ ]:
# fill missing fuel types with "Electric"
df["fuel_type"] = df["fuel_type"].fillna("Electric")


In [ ]:
# define valid fuel types
valid_fuel_types = ["Gasoline", "Diesel", "Hybrid", "Plug-In Hybrid", "Electric", "E85 Flex Fuel"]

# standardize text first
df["fuel_type"] = df["fuel_type"].str.strip().str.title()

# replace invalid or unsupported values
df["fuel_type"] = df["fuel_type"].apply(lambda x: x if x in valid_fuel_types else "Unknown")

df["fuel_type"].value_counts()




fuel_type
Gasoline          3309
Hybrid             194
Electric           170
E85 Flex Fuel      139
Diesel             116
Unknown             47
Plug-In Hybrid      34
Name: count, dtype: int64

In [ ]:
# display all Electric cars
print("=== Electric Cars ===")
print(df[df["fuel_type"] == "Electric"][["brand", "model", "fuel_type"]])

# display all Unknown fuel type cars
print("\n=== Unknown Fuel Type Cars ===")
print(df[df["fuel_type"] == "Unknown"][["brand", "model", "fuel_type"]])


=== Electric Cars ===
              brand                                     model fuel_type
9             Tesla                   Model X Long Range Plus  Electric
44            Lucid                         Air Grand Touring  Electric
68            Lucid                         Air Grand Touring  Electric
92           Rivian                     R1S Adventure Package  Electric
122          Rivian                     R1S Adventure Package  Electric
129           Lucid                                  Air Pure  Electric
155           Tesla                       Model 3 Performance  Electric
189           Tesla                       Model Y Performance  Electric
225           Tesla                              Model X Base  Electric
236        Polestar                          2 Launch Edition  Electric
253   Mercedes-Benz                            EQS 450 4MATIC  Electric
266   Mercedes-Benz                             EQS 450+ Base  Electric
298           Tesla               Model 3 

In [ ]:
# check accident
df["accident"].value_counts(dropna=False)


accident
None reported                             2910
At least 1 accident or damage reported     986
NaN                                        113
Name: count, dtype: int64

In [ ]:
# standardize accident
accident_mapping = {
    "None reported": "No",
    "At least 1 accident or damage reported": "Yes"
}

df["accident"] = df["accident"].map(accident_mapping)
df["accident"] = df["accident"].fillna("Unknown")
df["accident"].value_counts()


accident
No         2910
Yes         986
Unknown     113
Name: count, dtype: int64

In [ ]:
df["clean_title"] = df["clean_title"].fillna("Unknown")
df["clean_title"].value_counts()



clean_title
Yes        3413
Unknown     596
Name: count, dtype: int64

In [ ]:
pd.set_option('display.max_rows', None)
df["ext_col"].value_counts(dropna = False)



ext_col
Black                                      905
White                                      816
Gray                                       496
Silver                                     374
Blue                                       349
Red                                        261
Green                                       74
Brown                                       43
Gold                                        42
Beige                                       38
Orange                                      36
Yellow                                      29
–                                           15
Summit White                                14
Bright White Clearcoat                      14
Diamond Black                               14
Purple                                      12
Alpine White                                 8
Granite Crystal Clearcoat Metallic           7
Silver Ice Metallic                          7
Mythos Black Metallic                        6
Midni

In [ ]:
color_mapping = {
    "Summit White": "White",
    "Bright White Clearcoat": "White",
    "Diamond Black": "Black",
    "Purple": "Purple",
    "Alpine White": "White",
    "Granite Crystal Clearcoat Metallic": "Gray",
    "Silver Ice Metallic": "Silver",
    "Mythos Black Metallic": "Black",
    "Midnight Black Metallic": "Black",
    "Black Clearcoat": "Black",
    "Oxford White": "White",
    "Nightfall Gray Metallic": "Gray",
    "Lunar Rock": "Gray",
    "Quartzite Gray Metallic": "Gray",
    "Ebony Twilight Metallic": "Black",
    "Agate Black Metallic": "Black",
    "Fuji White": "White",
    "Glacial White Pearl": "White",
    "Black Sapphire Metallic": "Black",
    "Atomic Silver": "Silver",
    "Daytona Gray Pearl Effect": "Gray",
    "Snow White Pearl": "White",
    "Delmonico Red Pearlcoat": "Red",
    "Firecracker Red Clearcoat": "Red",
    "Soul Red Crystal Metallic": "Red",
    "Velvet Red Pearlcoat": "Red",
    "Snowflake White Pearl": "White",
    "Satin Steel Metallic": "Gray",
    "Lunar Silver Metallic": "Silver",
    "Obsidian Black Metallic": "Black",
    "Mythos Black": "Black",
    "Blu": "Blue",
    "Mosaic Black Metallic": "Black",
    "BLACK": "Black",
    "Ebony Black": "Black",
    "Midnight Black": "Black",
    "Majestic Black Pearl": "Black",
    "Flame Red Clearcoat": "Red",
    "Deep Black Pearl Effect": "Black",
    "Santorini Black Metallic": "Black",

}

df["ext_col_standard"] = df["ext_col"].replace(color_mapping)



In [ ]:
pd.set_option('display.max_rows', None)
df["ext_col_standard"].value_counts(dropna = False)



ext_col_standard
Black                                      972
White                                      876
Gray                                       513
Silver                                     388
Blue                                       352
Red                                        277
Green                                       74
Brown                                       43
Gold                                        42
Beige                                       38
Orange                                      36
Yellow                                      29
–                                           15
Purple                                      12
Majestic Black Pearl                         3
Flame Red Clearcoat                          3
Deep Black Pearl Effect                      3
Santorini Black Metallic                     3
Jet Black Mica                               3
Onyx Black                                   3
Platinum White Pearl                       